In [1]:
import numpy as np



## Day1

https://adventofcode.com/2024/day/1

In [2]:
#Part 1
data = np.loadtxt("./puzzle_files/aoc1.txt")
print(np.abs(np.sort(data[:,0]) - np.sort(data[:,1])).sum())

2815556.0


In [3]:
# Part 2
ans = 0
for k in data[:,0]:
    ans += k*(data[:,1] == k).sum()
print(ans)

23927637.0


## Day 2

https://adventofcode.com/2024/day/2

In [4]:
with open("./puzzle_files/aoc2.txt", "r") as f:
    lines = f.readlines()
    #print(lines)

def report_safe(dd):
    return (((dd == np.sort(dd)).all() or (dd == np.sort(dd)[::-1]).all()) and (np.abs(np.diff(dd)) <= 3).all() and (np.abs(np.diff(dd)) >= 1).all())

In [5]:
# Part 1
ans = 0
for line in lines:
    dd = np.array(line.split(), dtype=np.float64)
    if report_safe(dd):
        ans += 1

print(ans)

230


In [6]:
# Part 2
ans = 0
for line in lines:
    dd = np.array(line.split(), dtype=np.float64)
    if report_safe(dd):
        ans += 1
    else:
        for i in range(len(dd)):
            dd2 = np.delete(np.copy(dd), i)
            if report_safe(dd2):
                ans += 1
                break

print(ans)

301


## Day 3

https://adventofcode.com/2024/day/3

In [7]:
with open("./puzzle_files/aoc3.txt", "r") as f:
    lines = f.readlines()

In [8]:
import re
# Part 1
ans = 0
for line in lines:
    muls = re.findall("mul\(\d+,\d+\)", line)
    nums = np.array([mul[4:-1].split(',') for mul in muls], dtype=np.uint64)
    ans += (nums.prod(axis=1)).sum()
print(ans)

170778545.0


In [9]:
# Part 2
ans = 0
doflag = True
for line in lines:
    muls = re.findall("(mul\(\d+,\d+\))|(do\(\))|(don\'t\(\))", line)
    mul_array = []
    for mul in muls:
        if len(mul[1])+len(mul[2]) > 0:
            assert len(mul[0]) == 0
            if mul[1] == 'do()':
                doflag=True
            elif mul[2] == "don't()":
                doflag=False
            else:
                raise RunTimeError
        else:
            assert len(mul[0]) > 0
            if doflag:
                mul_array.append(mul[0][4:-1].split(','))

    ans += np.array(mul_array, dtype=np.uint64).prod(axis=1).sum()
print(ans)


82868252.0


## Day 4

https://adventofcode.com/2024/day/4

In [10]:
# Part 1
lines = np.loadtxt("./puzzle_files/aoc4.txt", dtype=str)
# make array
data = np.zeros((140, 140), dtype=str)

for i in range(len(lines)):
    data[i] = list(lines[i])

In [11]:
# Part 1
ans = 0
for line in data:
    muls = re.findall("(XMAS)", ''.join(line))
    ans += len(muls)
    muls = re.findall("(SAMX)", ''.join(line))
    ans += len(muls)

#transpose
for line in data.T:
    muls = re.findall("(XMAS)", ''.join(line))
    ans += len(muls)
    muls = re.findall("(SAMX)", ''.join(line))
    ans += len(muls)

#digonal
for i in range(-139, 140):
    line = np.diagonal(data, i)
    muls = re.findall("(XMAS)", ''.join(line))
    ans += len(muls)
    muls = re.findall("(SAMX)", ''.join(line))
    ans += len(muls)
    line = np.diagonal(np.fliplr(data), i)
    muls = re.findall("(XMAS)", ''.join(line))
    ans += len(muls)
    muls = re.findall("(SAMX)", ''.join(line))
    ans += len(muls)
print(ans)

2358


In [12]:
# Part 2
ans = 0
for i in range(1, 139):
    for j in range(1, 139):
        box = data[i-1:i+2, j-1:j+2]
        diag1 = ''.join(box.diagonal())
        diag2 = ''.join(np.fliplr(box).diagonal())
        if ((diag1 == "MAS") or (diag1 == "SAM")) and ((diag2 == "MAS") or (diag2 == "SAM")):
            ans += 1
print(ans)

1737


## Day 5

https://adventofcode.com/2024/day/5

In [13]:
with open("./puzzle_files/aoc5.txt", "r") as f:
    lines = f.readlines()

In [14]:
ordering_rules = []
update_order = []
for line in lines:
    if (len(line) == 6) and (line[2] == "|"):
        ordering_rules.append([int(value) for value in line[:-1].split("|")])
    elif len(line) > 1:
        update_order.append([int(value) for value in line[:-1].split(",")])
        
ordering_rules = np.array(ordering_rules)

In [15]:
def ord_less_then(A,B):
    return ((ordering_rules[:,0] == A) & (ordering_rules[:,1] == B)).any()

In [16]:
def bubble_sort(the_list):
    given_list = np.copy(the_list)
    for _ in range(len(given_list)):
        for i in range(1, len(given_list)):
            if not ord_less_then(given_list[i - 1], given_list[i]):
                given_list[i - 1], given_list[i] = given_list[i], given_list[i - 1]
    return given_list

In [17]:
# Part 1
ans = 0
for order in update_order:
    order = np.array(order)
    if (bubble_sort(bubble_sort(bubble_sort(order))) == order).all():
        ans += order[len(order)//2]
print(ans)

4959


In [18]:
#Part 2
ans = 0
for order in update_order:
    order = np.array(order)
    sorted_order = bubble_sort(order)
    if not (sorted_order == order).all():
        ans += sorted_order[len(order)//2]
print(ans)

4655


## Day 6
https://adventofcode.com/2024/day/6


In [19]:
with open("./puzzle_files/aoc6.txt", "r") as f:
    lines = f.read().splitlines() 
floor_map = np.zeros((len(lines), len(lines[0])), dtype=str)
for i, line in enumerate(lines):
    floor_map[i] = list(line)

In [20]:
def move(pos, direction):
    if direction == 0:
        obstacle = np.where(floor_map[:pos[0], pos[1]] == '#')[0]
        if len(obstacle) == 0:
            obstacle = -1
        else:
            obstacle = np.max(obstacle)
        presense_map[obstacle+1:pos[0], pos[1]] = True
        new_pos = [obstacle+1, pos[1]]
        return new_pos, (direction + 1)%4
    elif direction == 1:
        obstacle = np.where(floor_map[pos[0], pos[1]:] == '#')[0]
        if len(obstacle) == 0:
            obstacle = floor_map.shape[1]
        else:
            obstacle = np.min(obstacle)
        presense_map[pos[0], pos[1]:pos[1]+obstacle] = True
        new_pos = [pos[0], pos[1] + obstacle-1]
        return new_pos, (direction + 1)%4
    elif direction == 2:
        obstacle = np.where(floor_map[pos[0]:, pos[1]] == '#')[0]
        if len(obstacle) == 0:
            obstacle = floor_map.shape[0]
        else:
            obstacle = np.min(obstacle)
        presense_map[pos[0]:pos[0]+obstacle, pos[1]] = True
        new_pos = [pos[0] + obstacle-1, pos[1]]
        return new_pos, (direction + 1)%4
    elif direction == 3:
        obstacle = np.where(floor_map[pos[0], :pos[1]] == '#')[0]
        if len(obstacle) == 0:
            obstacle = -1
        else:
            obstacle = np.max(obstacle)
        presense_map[pos[0], obstacle+1:pos[1]] = True
        new_pos = [pos[0], obstacle+1]
        return new_pos, (direction + 1)%4
    else:
        raise RunTimeError("Invalid Direction")

def in_map(pos, direction):
    top_exit = ((pos[0] == 0) and (direction == 1))
    right_exit = ((pos[1] >= floor_map.shape[1]) and (direction == 2))
    down_exit = ((pos[0] >= floor_map.shape[0]) and (direction == 3))
    left_exit = ((pos[1] == 0) and (direction == 0))
    return not(top_exit or right_exit or down_exit or left_exit)

def looping(new_pos, new_dir, pos_history, dir_history):
    pos_a = np.array(pos_history)
    dir_a = np.array(dir_history)
    return ((new_pos[0] == pos_a[:,0]) & (new_pos[1] == pos_a[:,1]) &(new_dir == dir_a)).sum() == 1

In [21]:
# Part 1
presense_map = (floor_map == '^')
starting_pos = list(np.array(np.nonzero(presense_map)).flatten())
new_pos, new_dir = move(starting_pos, 0)
while in_map(new_pos, new_dir):
    new_pos, new_dir = move(new_pos, new_dir)
print(presense_map.sum())

5269


In [22]:
# Part 2 (Wrong)
from tqdm.notebook import trange
floor_map_main_copy = np.copy(floor_map)
presense_map_main_copy = np.copy(presense_map)
ans = 0
x_obs = np.where(presense_map_main_copy)[0]
y_obs = np.where(presense_map_main_copy)[1]
for i in trange(len(x_obs)):
    if x_obs[i] == starting_pos[0] and y_obs[i] == starting_pos[1]:
        continue
    floor_map = np.copy(floor_map_main_copy)
    presense_map = (floor_map == '^')
    starting_pos = list(np.array(np.nonzero(presense_map)).flatten())
    new_pos, new_dir = move(starting_pos, 0)
    pos_history = [starting_pos,new_pos]
    dir_history = [0, new_dir]
    floor_map[x_obs[i], y_obs[i]] = '#'
    while in_map(new_pos, new_dir):
        new_pos, new_dir = move(new_pos, new_dir)
        if looping(new_pos, new_dir, pos_history, dir_history):
            ans += 1
            break
        pos_history.append(new_pos)
        dir_history.append(new_dir)
print(ans)

  0%|          | 0/5269 [00:00<?, ?it/s]

1954


## Day 7

https://adventofcode.com/2024/day/7

In [23]:
with open("./puzzle_files/aoc7.txt", "r") as f:
    lines = f.read().splitlines() 

In [24]:
results = []
input_array = []
for line in lines:
    split_line = line.split(":")
    results.append(int(split_line[0]))
    input_array.append([int(k) for k in split_line[1].split()])

In [28]:
import itertools
def find_operation_combinations(N, operators="+*|"):
    return itertools.product(operators, repeat=N-1)

In [26]:
def evaluate(inputtt, operations):
    ans = inputtt[0]
    for i in range(len(operations)):
        if operations[i] == "|":
            ans = int(f"{ans}{inputtt[i+1]}")
        else:
            ans = eval(f"ans {operations[i]} {inputtt[i+1]}")
    return ans

In [29]:
ans = 0
from tqdm.notebook import tqdm
for inputs, result in tqdm(zip(input_array, results), total=len(input_array)):
    for operation in find_operation_combinations(len(inputs), operators="+*"):
        if evaluate(inputs, operation) == result:
            ans += result
            break
print(ans)

  0%|          | 0/850 [00:00<?, ?it/s]

267566105056


In [27]:
ans = 0
from tqdm.notebook import tqdm
for inputs, result in tqdm(zip(input_array, results), total=len(input_array)):
    for operation in find_operation_combinations(len(inputs)):
        if evaluate(inputs, operation) == result:
            ans += result
            break
print(ans)

  0%|          | 0/850 [00:00<?, ?it/s]

116094961956019
